In [5]:
import csv
import os
import time
from itertools import permutations
from pathlib import Path

In [6]:
def otimiza_sprints(dados, cap_sprint):
    '''

    :param dados:
    :param cap_sprint:
    :return:
    '''

    # armazena o melhor cenário
    melhor_cenario = {'requisitos':[], 'valores':[], 'pesos':[]}

    #armazena o melhor caso de requisitos agrupados por sprint
    melhor_retorno={}

    #faz a permutação dos requisitos
    perm = permutations(dados['requisitos'])

    for requisitos_permutados in (perm):

        #iniciaclização de variáveis

        #contador com o número de sprints
        num_sprints = 1

        # controle temporário de requisitos agrupados por sprint
        retorno = {}
        retorno.__setitem__('Sprint ' + str(num_sprints), [])

        #agrupa pesos por sprint
        peso_por_sprint = []
        pesos_somados = 0

        #agrupa valores por sprint
        valor_por_sprint = []
        valores_somados=0

        #executa a força bruta percorrendo todos os casos permutados
        for requisito in requisitos_permutados:

            #só executa caso a capacidade da sprint seja maior que a soma dos pesos
            if (cap_sprint>=pesos_somados):

                peso=dados['pesos'][dados['requisitos'].index(requisito)]
                valor= dados['valores'][dados['requisitos'].index(requisito)]

                #enquanto a soma dos pesos não foir maior que a capacidade, soma os pesos
                if peso + pesos_somados <= cap_sprint:
                    pesos_somados+=peso
                    valores_somados+=valor
                #finaliza uma sprint e cria a próxima
                else:
                    num_sprints+=1
                    retorno.__setitem__('Sprint ' + str(num_sprints), [])
                    peso_por_sprint.append(pesos_somados)
                    valor_por_sprint.append(valores_somados)
                    pesos_somados=peso
                    valores_somados = valor

            #atualiza o controle temporário de lista de requisitos por sprint
            if requisito not in retorno['Sprint ' + str(num_sprints)]:
                retorno['Sprint ' + str(num_sprints)].append(requisito)

        #verifica se há resíduo
        if len(peso_por_sprint)<len(requisitos_permutados):
            peso_por_sprint.append(pesos_somados)
            valor_por_sprint.append(valores_somados)

            # atualiza o controle temporário de lista de requisitos por sprint
            if requisito not in retorno['Sprint ' + str(num_sprints)]:
                retorno['Sprint ' + str(num_sprints)].append(requisito)

        #verifica se o cenário atual é o melhor cenário e armazena
        if valor_por_sprint > melhor_cenario['valores']:
            melhor_cenario['valores'] = valor_por_sprint
            melhor_cenario['pesos'] = peso_por_sprint
            melhor_cenario['requisitos'] = requisitos_permutados
            melhor_retorno = retorno


    print('Melhor cenário: ',melhor_cenario)
    print('Melhor retorno: ', melhor_retorno)

    return melhor_retorno

In [7]:
def imprime_resultados_csv(arquivo, dados_entrada, resultado_alocacao):
    """
    Esta função imprime a solução para o problema em um arquivo CSV.
    Cada linha do arquivo contem a informações sobre asprint, requisito, valor e peso do requisito.
    O formato da linha do arquivo gerado é: sprint,requisito,valor_req,peso_req

    :param arquivo: o arquivo no qual as informações da solução serão armazenadas.
    :param dados_entrada: os dados de entrada utilizados no problema.
                            Trata-se de um objeto dicionário, com formato:
                                {
                                    requisitos: [req_1, ..., req_n],
                                    valores: [val_1, ..., val_n],
                                    pesos: [peso_1, ..., peso_n]
                                 }
    :param resultado_alocacao: a solução para o problema especificado, ou seja,
                                o relacionamento entre sprints e requisitos.
                                Trata-se de um objeto dicionário, com formato:
                                    {
                                        sprint1: [req1, ..., req_n],
                                        ...,
                                        sprint_n: [req1, ..., req_n]
                                     }
    :return: True se houver solução ótima e o arquivo for gerado com sucesso, False caso contrário.
    """
    # os nomes das colunas que constarão no arquivo CSV
    nomes_campos_arquivo = ['sprint', 'requisito', 'valor_req', 'peso_req']

    # lista que conterá as informações a serem gravas em arquivo (sprint, requisito, valor e peso)
    informacoes_arquivo = []

    # itera sobre cada sprint da solução
    for sprint in resultado_alocacao.keys():
        # recupera os requisitos que foram alocados nesta sprint
        reqs_sprint = resultado_alocacao[sprint]

        # itera sobre cada requisito alocada nesta sprint
        for requisito in reqs_sprint:
            idx = dados_entrada['requisitos'].index(requisito)
            val_req = dados_entrada['valores'][idx]
            peso_req = dados_entrada['pesos'][idx]

            # detalhe de cada linha do arquivo: sprint, requisito, valor do requisito e peso do requisito
            detalhe_linha = {
                                nomes_campos_arquivo[0]: sprint,
                                nomes_campos_arquivo[1]: requisito,
                                nomes_campos_arquivo[2]: val_req,
                                nomes_campos_arquivo[3]: peso_req
                             }

            informacoes_arquivo.append(detalhe_linha)

    # realiza a escrita dos dados no arquivo especificado
    with open(arquivo, 'w',  newline='') as csvfile_output:
        writer = csv.DictWriter(csvfile_output, fieldnames=nomes_campos_arquivo)
        writer.writeheader()
        writer.writerows(informacoes_arquivo)

    return True

In [8]:
# recupera o caminho completo do diretório de dados de entrada
dir_dados_entrada = str(Path(os.getcwd()).parents[0]) + '\data\input'

# recupera o caminho completo do diretório de dados de resultado
dir_dados_saida = str(Path(os.getcwd()).parents[0]) + '\data\output'

# cria o diretório de saida que armazenará os resultados
try:
    os.mkdir(dir_dados_saida)
except FileExistsError:
    # se o diretório já existir, não é preciso fazer nada
    pass

# percorre cada arquivo presente no diretório de dados de entrada
for arquivo_entrada in Path(dir_dados_entrada).iterdir():
    # lista que armazenará os dados de cada arquivo
    dados_a_otimizar = {}
    requisitos = []
    valores = []
    pesos = []

    if arquivo_entrada.is_file() and arquivo_entrada.suffix == '.csv':
        # faz a leitura do CSV
        with open(arquivo_entrada, newline='') as csvfile:
            leitor = csv.DictReader(csvfile)
            for linha in leitor:
                requisitos.append(linha['requisito'])
                valores.append(float(linha['valor']))
                pesos.append(float(linha['peso']))

        dados_a_otimizar['requisitos'] = requisitos
        dados_a_otimizar['valores'] = valores
        dados_a_otimizar['pesos'] = pesos

    # define a capacidade total necessária para implementar todos os requisitos do backlog
    capacidade_necessaria_backlog = 0
    for peso in dados_a_otimizar['pesos']:
        capacidade_necessaria_backlog += peso

    # define o arquivo de saída para o resultado
    arquivo_resultado = dir_dados_saida + '\\resultado-BRTF-req-' + str(len(dados_a_otimizar['requisitos'])) + '.csv'

    tempo_inicial = time.time()

    # chama função que realiza a otimização
    alocacao_sprints = otimiza_sprints(dados_a_otimizar, 480)

    tempo_final = time.time()

    # escreve os resultados em disco
    imprime_resultados_csv(arquivo_resultado, dados_a_otimizar, alocacao_sprints)

    print(f'Tempo de execução para {len(requisitos)} requisitos: {(tempo_final - tempo_inicial)} segundos')

Melhor cenário:  {'requisitos': ('Requisito_1', 'Requisito_4', 'Requisito_7', 'Requisito_8', 'Requisito_9', 'Requisito_10', 'Requisito_2', 'Requisito_5', 'Requisito_6', 'Requisito_3'), 'valores': [195.0, 69.0, 10.0], 'pesos': [472.0, 372.0, 131.0]}
Melhor retorno:  {'Sprint 1': ['Requisito_1', 'Requisito_4', 'Requisito_7', 'Requisito_8', 'Requisito_9', 'Requisito_10'], 'Sprint 2': ['Requisito_2', 'Requisito_5', 'Requisito_6'], 'Sprint 3': ['Requisito_3']}
Tempo de execução para 10 requisitos: 42.887258768081665 segundos


KeyboardInterrupt: 